In [1]:
%load_ext lab_black

In [2]:
from pathlib import Path

cmds = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k""".split(
    "\n"
)
# cmds = Path("data/07.txt").read_text().split("\n")

In [ ]:
root = []
current = root

for cmd in cmds[1:]:
    tokens = cmd.split()

    if tokens[0] == "$":
        if tokens[1] == "cd":
            if tokens[2] == "..":
                current = current.parent
            else:
                current = current.children[tokens[2]]
    else:
        size, name = tokens
        if size == "dir":
            node = Folder(name)
        else:
            node = File(name, size=int(size))
        current.add_child(node)

In [88]:
class Node:
    def __init__(self, name: str):
        self.name = name
        self.parent = None


class File(Node):
    def __init__(self, name: str, size: int):
        self.size = size
        super().__init__(name)

    def get_size(self):
        return self.size


class Folder(Node):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.children = {}

    def get_size(self):
        return sum([n.get_size() for n in self.children.values()])

    def add_child(self, node: Node):
        self.children[node.name] = node
        node.parent = self

In [89]:
root = Folder("/")
current = root

for cmd in cmds[1:]:
    tokens = cmd.split()

    if tokens[0] == "$":
        if tokens[1] == "cd":
            if tokens[2] == "..":
                current = current.parent
            else:
                current = current.children[tokens[2]]
    else:
        size, name = tokens
        if size == "dir":
            node = Folder(name)
        else:
            node = File(name, size=int(size))
        current.add_child(node)

In [90]:
def bfs(node):
    queue = [node]
    while len(queue) > 0:
        node = queue.pop()
        yield node
        if isinstance(node, Folder):
            queue.extend(list(node.children.values()))

In [94]:
S = 0
for n in bfs(root):
    if isinstance(n, Folder) and n.get_size() <= 100_000:
        S += n.get_size()
S

1086293

In [98]:
x0 = 30000000 - (70000000 - root.get_size())

min(
    [
        n.get_size()
        for n in bfs(root)
        if isinstance(n, Folder) and n.get_size() - x0 >= 0
    ]
)

366028